<a href="https://colab.research.google.com/github/pyijiezhang/cold-posteriors-and-pacbayes/blob/main/Yijie's_Variational_NNs_Keras_TFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-Up



In [1]:
!pip uninstall tensorflow -y

In [2]:
!pip install tensorflow==2.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-2.8.3-cp38-cp38-manylinux2010_x86_64.whl (498.4 MB)


In [3]:
!pip uninstall import tensorflow_probability -y

Found existing installation: tensorflow-probability 0.16.0
Uninstalling tensorflow-probability-0.16.0:
  Successfully uninstalled tensorflow-probability-0.16.0


In [4]:
!pip install tensorflow_probability==0.16.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_probability-0.16.0-py2.py3-none-any.whl (6.3 MB)


In [5]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [6]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [37]:
import datetime
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, Lambda

from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

from tqdm import tqdm


from __future__ import absolute_import, division, print_function, unicode_literals

In [8]:
assert(tf.test.gpu_device_name())
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.

In [9]:
from google.colab import files
import pickle

In [10]:
print(tf.__version__)

2.8.3


# Data

In [11]:
num_classes = 10

def load_data(data_set):
  if data_set=='fashion_mnist':
    (x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()
  else:
    (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
  

  tf.keras.utils.set_random_seed(123)
  a = np.random.permutation(60000) #60000 original
  x_train = x_train[a,...]
  y_train = y_train[a]
  # Add a new axis
  x_train = x_train[..., np.newaxis]
  x_test = x_test[..., np.newaxis]


  # Convert class vectors to binary class matrices.
  y_train = to_categorical(y_train, num_classes)
  y_test = to_categorical(y_test, num_classes)

  # Data normalization
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255

  return x_train, y_train, x_test, y_test

# Prior Definition

Define functions to initialize different types of prior distributions.

In [12]:
from tensorflow_probability.python.distributions import normal as normal_lib
from tensorflow_probability.python.distributions import independent as independent_lib
from tensorflow_probability.python.distributions import kullback_leibler as kl_lib

def make_normal_prior(scale):
  """
  Defines a function that returns a Gaussian prior with the given standard
  deviation.
  """
  def _fn(dtype, shape, name, trainable,add_variable_fn):
    del name, trainable, add_variable_fn   # unused
    dist = normal_lib.Normal(loc=tf.zeros(shape, dtype), 
                             scale=dtype.as_numpy_dtype(scale))
    batch_ndims = tf.size(dist.batch_shape_tensor())
    return independent_lib.Independent(dist, 
                                       reinterpreted_batch_ndims=batch_ndims)

  return _fn

# Posteriors Definition

Define posterior distributions.

In [13]:
import tensorflow.compat.v1 as tf1

mean_field_init_untransformed_scale = -7.0

# TODO(trandustin): Remove need for this boilerplate code.
def mean_field_fn(empirical_bayes=False,
                  initializer=tf1.initializers.he_normal()):
  """Constructors for Gaussian prior and posterior distributions.
  Args:
    empirical_bayes (bool): Whether to train the variance of the prior or not.
    initializer (tf1.initializer): Initializer for the posterior means.
  Returns:
    prior, posterior (tfp.distribution): prior and posterior
    to be fed into a Bayesian Layer.
  """

  def prior(dtype, shape, name, trainable, add_variable_fn):
    """Returns the prior distribution (tfp.distributions.Independent)."""
    softplus_inverse_scale = np.log(np.exp(1.) - 1.)

    istrainable = add_variable_fn(
        name=name + '_istrainable',
        shape=(),
        initializer=tf1.constant_initializer(1.),
        dtype=dtype,
        trainable=False)

    untransformed_scale = add_variable_fn(
        name=name + '_untransformed_scale',
        shape=(),
        initializer=tf1.constant_initializer(softplus_inverse_scale),
        dtype=dtype,
        trainable=empirical_bayes and trainable)
    scale = (
        np.finfo(dtype.as_numpy_dtype).eps +
        tf.nn.softplus(untransformed_scale * istrainable + (1. - istrainable) *
                       tf1.stop_gradient(untransformed_scale)))
    loc = add_variable_fn(
        name=name + '_loc',
        shape=shape,
        initializer=tf1.constant_initializer(0.),
        dtype=dtype,
        trainable=False)
    dist = tfp.distributions.Normal(loc=loc, scale=scale)
    dist.istrainable = istrainable
    dist.untransformed_scale = untransformed_scale
    batch_ndims = tf1.size(input=dist.batch_shape_tensor())
    return tfp.distributions.Independent(dist,
                                         reinterpreted_batch_ndims=batch_ndims)

  def posterior(dtype, shape, name, trainable, add_variable_fn):
    """Returns the posterior distribution (tfp.distributions.Independent)."""
    untransformed_scale = add_variable_fn(
        name=name + '_untransformed_scale',
        shape=shape,
        initializer=tf1.initializers.random_normal(
            mean=mean_field_init_untransformed_scale, stddev=0.1),
        dtype=dtype,
        trainable=trainable)
    scale = (
        np.finfo(dtype.as_numpy_dtype).eps +
        tf.nn.softplus(untransformed_scale))
    loc = add_variable_fn(
        name=name + '_loc',
        shape=shape,
        initializer=initializer,
        dtype=dtype,
        trainable=trainable)
    dist = tfp.distributions.Normal(loc=loc, scale=scale)
    dist.untransformed_scale = untransformed_scale
    batch_ndims = tf1.size(input=dist.batch_shape_tensor())
    return tfp.distributions.Independent(dist,
                                         reinterpreted_batch_ndims=batch_ndims)

  return prior, posterior

prior_fn, posterior_fn = mean_field_fn(empirical_bayes=False)

In [14]:
from tensorflow_probability.python.distributions import deterministic as deterministic_lib

def make_normal_posterior(
    is_singular=False,
    loc_initializer=tf1.initializers.random_normal(stddev=0.1),
    untransformed_scale_initializer=tf1.initializers.random_normal(
        mean=-3., stddev=0.1),
    loc_regularizer=None,
    untransformed_scale_regularizer=None,
    loc_constraint=None,
    untransformed_scale_constraint=None):

  loc_scale_fn = tfp.layers.util.default_loc_scale_fn(
      is_singular=is_singular,
      loc_initializer=loc_initializer,
      untransformed_scale_initializer=untransformed_scale_initializer,
      loc_regularizer=loc_regularizer,
      untransformed_scale_regularizer=untransformed_scale_regularizer,
      loc_constraint=loc_constraint,
      untransformed_scale_constraint=untransformed_scale_constraint)
  def _fn(dtype, shape, name, trainable, add_variable_fn):
    loc, scale = loc_scale_fn(dtype, shape, name, trainable, add_variable_fn)
    if scale is None:
      dist = deterministic_lib.Deterministic(loc=loc)
    else:
      dist = normal_lib.Normal(loc=loc, scale=scale)
    batch_ndims = tf.size(dist.batch_shape_tensor())
    return independent_lib.Independent(
        dist, reinterpreted_batch_ndims=batch_ndims)
  return _fn




In [15]:
import tensorflow.compat.v1 as tf1
from tensorflow_probability.python.distributions import deterministic as deterministic_lib

def default_mean_field_mixture_fn(
    is_singular=False,
    loc_initializer=tf1.initializers.random_normal(stddev=0.1),
    untransformed_scale_initializer=tf1.initializers.random_normal(
        mean=-3., stddev=0.1),
    loc_regularizer=None,
    untransformed_scale_regularizer=None,
    loc_constraint=None,
    untransformed_scale_constraint=None):
 
  loc_scale_fn = tfp.layers.util.default_loc_scale_fn(
      is_singular=is_singular,
      loc_initializer=loc_initializer,
      untransformed_scale_initializer=untransformed_scale_initializer,
      loc_regularizer=loc_regularizer,
      untransformed_scale_regularizer=untransformed_scale_regularizer,
      loc_constraint=loc_constraint,
      untransformed_scale_constraint=untransformed_scale_constraint)
  def _fn(dtype, shape, name, trainable, add_variable_fn):
    loc, scale = loc_scale_fn(dtype, shape, name, trainable, add_variable_fn)
    #loc_2, scale_2 = loc_scale_fn(dtype, shape, name, trainable, add_variable_fn)
    if scale is None:
      dist = deterministic_lib.Deterministic(loc=loc)
    else:
      dist = tfp.distributions.Normal(loc=loc, scale=scale)
      #dist2 = tfp.distributions.Normal(loc=tf.stop_gradient(loc_2), scale=scale_2)

    batch_ndims = tf.size(dist.batch_shape_tensor())

    return tfp.distributions.Mixture(
        cat=tfp.distributions.Categorical(probs=[0.5, 0.5]),
        components=[
            tfp.distributions.Independent(dist,
                            reinterpreted_batch_ndims=batch_ndims),
            tfp.distributions.Independent(tfp.distributions.Normal(
                loc=tf.zeros(shape, dtype=dtype), 
                scale=1.0*tf.ones(shape, dtype=dtype)),
                            reinterpreted_batch_ndims=batch_ndims)],
        name='spike_and_slab')

  return _fn


# Model

## Set-up

In [16]:
def conv2d(filters, kernel_size, padding='SAME', l2_val=2e-4):
    return tf.keras.layers.Conv2D(
          filters,
          kernel_size,
          padding=padding,
          activation=tf.nn.relu,
          bias_regularizer=tf.keras.regularizers.L2(l2_val),
          kernel_regularizer=tf.keras.regularizers.L2(l2_val))

def max_pool():
    return tf.keras.layers.MaxPooling2D(
        pool_size=[2, 2],
        strides=[2, 2],
        #padding='same',
    )
def avg_pool():
    return tf.keras.layers.AveragePooling2D(
        pool_size=[2, 2],
        strides=[2, 2],
        #padding='same',
    )

def dense(units, activation, l2_val=2e-4):
    return tf.keras.layers.Dense(
        units,
        activation,
        bias_regularizer=tf.keras.regularizers.L2(l2_val),
        kernel_regularizer=tf.keras.regularizers.L2(l2_val))


In [17]:
def conv2d_variational(filters, kernel_size, strides=(1, 1), padding='SAME', kernel_posterior_fn=None, kernel_prior_fn=None, bias_prior_fn=None, kernel_divergence_fn=None):
    """Convenience wrapper for conv layers."""
    if bias_prior_fn==None:
      bias_prior_fn = kernel_prior_fn
    
    return tfp.layers.Convolution2DReparameterization(
          filters,
          kernel_size,
          padding=padding,
          activation=tf.nn.relu,
          kernel_posterior_fn=kernel_posterior_fn,
          kernel_prior_fn=kernel_prior_fn,
          kernel_divergence_fn=kernel_divergence_fn,
          bias_posterior_fn = kernel_posterior_fn,
          bias_prior_fn = bias_prior_fn,
          bias_divergence_fn=kernel_divergence_fn)


def dense_variational(units, activation, kernel_posterior_fn=None, kernel_prior_fn=None, bias_prior_fn=None, kernel_divergence_fn=None):
    if bias_prior_fn==None:
      bias_prior_fn = kernel_prior_fn

    return tfp.layers.DenseReparameterization(
        units,
        activation,
        kernel_posterior_fn=kernel_posterior_fn,
        kernel_prior_fn=kernel_prior_fn,
        kernel_divergence_fn=kernel_divergence_fn,
        bias_posterior_fn = kernel_posterior_fn,
        bias_prior_fn = bias_prior_fn,
        bias_divergence_fn=kernel_divergence_fn)

## LeNet

### Variational

In [18]:
class LeNet_Variational(Sequential):
  def __init__(self, input_shape, nb_classes,
               kernel_posterior_fn=make_normal_posterior(),
               kernel_prior_fn=make_normal_prior(1.0),
               kernel_divergence_fn=lambda q, p, ignore: kl_lib.kl_divergence(q, p)/(x_train.shape[0]*lamb),
               gamma=1.0):
    
    super().__init__()

    self.weight_layers = [0, 2, 4, 6, 7]

    self.add(tf.keras.layers.Input(shape=input_shape))

    self.add(conv2d_variational(6, 5, padding = "same", kernel_posterior_fn=kernel_posterior_fn, 
                                kernel_prior_fn=kernel_prior_fn, 
                                kernel_divergence_fn=kernel_divergence_fn))
    self.add(avg_pool())

    self.add(conv2d_variational(16, 5, padding="same", kernel_posterior_fn=kernel_posterior_fn, 
                                kernel_prior_fn=kernel_prior_fn, 
                                kernel_divergence_fn=kernel_divergence_fn))
    self.add(avg_pool())

    self.add(Flatten())


    self.add(dense_variational(120, tf.nn.relu, 
                               kernel_posterior_fn=kernel_posterior_fn, 
                               kernel_prior_fn=kernel_prior_fn, 
                               kernel_divergence_fn=kernel_divergence_fn))


    self.add(dense_variational(84, tf.nn.relu, 
                               kernel_posterior_fn=kernel_posterior_fn, 
                               kernel_prior_fn=kernel_prior_fn, 
                               kernel_divergence_fn=kernel_divergence_fn))

    self.add(dense_variational(nb_classes, None, 
                               kernel_posterior_fn=kernel_posterior_fn, 
                               kernel_prior_fn=kernel_prior_fn, 
                               kernel_divergence_fn=kernel_divergence_fn))
    
    self.add(Lambda(lambda x: x*gamma))

In [19]:
class LeNet_Variational_Large(Sequential):
  def __init__(self, input_shape, nb_classes,
               kernel_posterior_fn=make_normal_posterior(),
               kernel_prior_fn=make_normal_prior(1.0),
               kernel_divergence_fn=lambda q, p, ignore: kl_lib.kl_divergence(q, p)/(x_train.shape[0]*lamb),
               gamma=1.0):
    
    super().__init__()

    self.weight_layers = [0, 2, 4, 6, 7]

    self.add(tf.keras.layers.Input(shape=input_shape))

    self.add(conv2d_variational(6, 5, kernel_posterior_fn=kernel_posterior_fn, 
                                kernel_prior_fn=kernel_prior_fn, 
                                kernel_divergence_fn=kernel_divergence_fn))
    self.add(max_pool())

    self.add(conv2d_variational(16, 5, kernel_posterior_fn=kernel_posterior_fn, 
                                kernel_prior_fn=kernel_prior_fn, 
                                kernel_divergence_fn=kernel_divergence_fn))
    self.add(max_pool())

    self.add(conv2d_variational(120, 5, kernel_posterior_fn=kernel_posterior_fn, 
                                kernel_prior_fn=kernel_prior_fn, 
                                kernel_divergence_fn=kernel_divergence_fn))

    self.add(Flatten())

    self.add(dense_variational(84, tf.nn.relu, 
                               kernel_posterior_fn=kernel_posterior_fn, 
                               kernel_prior_fn=kernel_prior_fn, 
                               kernel_divergence_fn=kernel_divergence_fn))

    self.add(dense_variational(nb_classes, None, 
                               kernel_posterior_fn=kernel_posterior_fn, 
                               kernel_prior_fn=kernel_prior_fn, 
                               kernel_divergence_fn=kernel_divergence_fn))
    
    self.add(Lambda(lambda x: x*gamma))
     

In [20]:
x_train, y_train, x_test, y_test = load_data('mnist')
lamb=1
model_variational = LeNet_Variational_Large(x_train[0].shape, num_classes)
model_variational.summary()

/usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/layers/util.py:95: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  loc = add_variable_fn(
/usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/layers/util.py:105: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  untransformed_scale = add_variable_fn(


Model: "le_net__variational__large"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_reparameterization (  (None, 28, 28, 6)        312       
 Conv2DReparameterization)                                       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 6)        0         
 )                                                               
                                                                 
 conv2d_reparameterization_1  (None, 14, 14, 16)       4832      
  (Conv2DReparameterization)                                     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 conv2d_reparameterization_2  (None, 7, 

# Training Set-up

In [38]:
cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.0)

@tf.function
def ece(y_true,y_pred):
  return tfp.stats.expected_calibration_error(10,logits=y_pred, labels_true=tf.argmax(y_true,axis=1))

# Place the logs in a timestamped subdirectory
# This allows to easy select different training runs
# In order not to overwrite some data, it is useful to have a name with a timestamp
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Specify the callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_categorical_crossentropy", patience=10, restore_best_weights=True)

adam_opt = tf.keras.optimizers.Adam(0.001)

def schedule(epoch, lr):
  if (epoch<25):
    return 0.001
  elif epoch<50:
    return 0.0001
  else:
    return 0.00001

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule)



In [39]:
class BMA_Model(tf.keras.Model):

  def __init__(self, model, mc_samples):
    super().__init__()
    self.model = model
    self.mc_samples = mc_samples
  
  def call(self, inputs, training=False):
    list_models =[self.model(inputs) for i in range(self.mc_samples)]
    stack_models = tf.stack(list_models,axis=1)
    return tfp.math.reduce_logmeanexp(stack_models,axis=1)

# Variational Learning

In [55]:
# experiment settings
data_use="mnist"
model_used="lenet"
num_posterior_samples=10
n_epochs=100
batch_size=100
lr=0.001

seeds=[15]#,24]

# lambs for likelihood tempering
lambs=np.linspace(0.5, 3.5, 7)
#lambs = [1.0]

# label_smoothing
# label_smoothing=0.0
# cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=label_smoothing)

# label noise
label_noises=[0.0] #0.1,0.2,0.3]

# smooth softmax
gammas=[1.0]#,5.0,10.0]

# data augmentation:
#   - None: No data augmentation
#   - Standard: 
#   - Replacement
#   - Noise
#d_augmentation = ["Standard", "Replacement", "Noise"]
d_augmentation = ["Replacement"]

# prior
#prior_scales=[1.0, 0.1, 0.01]
prior_scales=[1.0]


In [56]:
# x_train_fashion, y_train_fashion, x_test_fashion, y_test_fashion = load_data('fashion_mnist')
x_train_mnist, y_train_mnist, x_test_mnist, y_test_mnist = load_data('mnist')

x_train_mnist = x_train_mnist[:10000]
y_train_mnist = y_train_mnist[:10000]

In [57]:
model_variational=[]
for prior_scale in prior_scales: 
  for gamma in gammas:
    for label_noise in label_noises:
      for da in d_augmentation:

        setting_name=f"prior_scale_{prior_scale}_label_noise_{label_noise}_smooth_softmax_{gamma}_data_augmentation_{da}" 

        x_train = x_train_mnist
        y_train = y_train_mnist
        x_test = x_test_mnist
        y_test = y_test_mnist

        input_shape=x_train[0].shape

        if label_noise > 0.0:
          import random
          # train
          ind_noisy=random.sample(range(1, y_train.shape[0]), int(y_train.shape[0]*label_noise))
          for ind in ind_noisy:
            false_label=random.sample(range(1,y_train.shape[1]),1)[0]
            while y_train[ind, false_label] != 1.0:
              temp=np.zeros((y_train.shape[1]))
              temp[false_label]=1.0
              y_train[ind, :]=temp
              break
            else:
              false_label=random.sample(range(1,y_train.shape[1]),1)[0]
          # test
          ind_noisy=random.sample(range(1, y_test.shape[0]), int(y_test.shape[0]*label_noise))
          for ind in ind_noisy:
            false_label=random.sample(range(1,y_test.shape[1]),1)[0]
            while y_test[ind, false_label] != 1.0:
              temp=np.zeros((y_test.shape[1]))
              temp[false_label]=1.0
              y_test[ind, :]=temp
              break
            else:
              false_label=random.sample(range(1,y_test.shape[1]),1)[0]

        from tensorflow import keras
        from tensorflow.keras import layers

        if da == "None":

            train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            train_loader = train_loader.batch(batch_size)

        if da == "Standard":
            # Create a data augmentation stage with horizontal flipping, rotations, zooms
            data_augmentation = keras.Sequential(
                [
                    layers.RandomFlip("horizontal"),
                    layers.RandomRotation(0.1),
                    layers.RandomZoom(0.1),
                ]
            )
            # Create a tf.data pipeline of augmented images (and their labels)
            train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            train_loader = train_loader.batch(batch_size).map(lambda x, y: (data_augmentation(x), y))


        if da == "Noise":

            rng = np.random.default_rng(1234)
            n_features = x_train.shape[1]*x_train.shape[2]*x_train.shape[3]
            idx = np.random.choice(np.arange(0, n_features, 1), n_features // 5 * 4, replace = False)
            perm = rng.permutation(idx)

            aux =  x_train.reshape(x_train.shape[0], -1)
            aux[:, idx] = aux[:, perm]

            x_train = aux.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3])

            # Create a tf.data pipeline of augmented images (and their labels)
            train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            train_loader = train_loader.batch(batch_size)

        if da == "Replacement":

            rng = np.random.default_rng(1234)

            idx = np.random.choice(np.arange(0, x_train.shape[0], 1), int(x_train.shape[0]*0.05), replace= False)
            idx = np.random.choice(idx, x_train.shape[0], replace= True)

            # Create a tf.data pipeline of augmented images (and their labels)
            train_loader = tf.data.Dataset.from_tensor_slices((x_train[idx], y_train[idx]))
            train_loader = train_loader.batch(batch_size)
        
        
        input_shape = x_train.shape[1:]
        results={}

        for seed in seeds:
          print(seed)
          results[seed]={} # a dict, with the structure, dict_results[lamb]=[log_ps_train, log_ps_test, metrics_bma]
          for lamb in tqdm(lambs):
            
            print(lamb)
            results[seed][lamb]=[]

            tf.keras.utils.set_random_seed(seed)

            model_variational = LeNet_Variational_Large(input_shape, num_classes,
                                                      kernel_prior_fn=make_normal_prior(prior_scale),
                                                      kernel_divergence_fn=lambda q, p, ignore: kl_lib.kl_divergence(q, p)/(x_train.shape[0]*lamb),
                                                      gamma=gamma)

            model_variational.compile(optimizer=tf.keras.optimizers.Adam(lr),
                            loss = cce,
                            metrics=['accuracy', cce]
                            )


            model_variational.fit(train_loader, 
                    epochs=n_epochs, 
                    #validation_data = (x_test, y_test),
                    # workers=8,
                    verbose = 0)
            

            log_ps_train=[]
            log_ps_test=[]
            metrics_gibbs_train=[]
            metrics_gibbs_test=[]
            metrics_bayes_train=[]
            metrics_bayes_test=[]

            # loop through posterior samples
            for i in range(num_posterior_samples):
              
              # compute the categorical dist. and log_p for train data
              x_train_augmented = np.concatenate([x for x, y in train_loader.as_numpy_iterator()])
              y_train_augmented = np.concatenate([y for x, y in train_loader.as_numpy_iterator()])
              p_train=tfp.distributions.Categorical(logits=model_variational(x_train_augmented))
              log_p_train=p_train.log_prob(tf.argmax(y_train_augmented, axis=1))

              log_ps_train.append(log_p_train)

              # compute the categorical dist. and log_p for test data
              p_test=tfp.distributions.Categorical(logits=model_variational(x_test)) 
              log_p_test=p_test.log_prob(tf.argmax(y_test, axis=1))
              log_ps_test.append(log_p_test)



              # compute gibbs-based metrics
              metrics_gibbs_train.append(model_variational.evaluate(x_train, y_train, batch_size=batch_size))
              metrics_gibbs_test.append(model_variational.evaluate(x_test, y_test, batch_size=batch_size))

            # compute bayes-based metric
            bma_model=BMA_Model(model_variational, num_posterior_samples)
            bma_model.compile(metrics=['accuracy', cce])
            metrics_bayes_train.append(bma_model.evaluate(x_train, y_train, batch_size=batch_size))
            metrics_bayes_test.append(bma_model.evaluate(x_test, y_test, batch_size=batch_size))

            !nvidia-smi
            
            # save log_p for train data
            results[seed][lamb].append(tf.stack(log_ps_train,axis=1).numpy())
            # save log_p for test data 
            results[seed][lamb].append(tf.stack(log_ps_test,axis=1).numpy())
            # save gibbs-based metric
            results[seed][lamb].append(metrics_gibbs_train)
            results[seed][lamb].append(metrics_gibbs_test)
            # save bayes-based metric
            results[seed][lamb].append(metrics_bayes_train)
            results[seed][lamb].append(metrics_bayes_test)

        with open(f'{setting_name}.pickle', 'wb') as handle:
          pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

15


  0%|          | 0/7 [00:00<?, ?it/s]

0.5
100/100 [==============================] - 1s 6ms/step - loss: 39.0444 - accuracy: 0.9287 - categorical_crossentropy: 0.5226
Tue Apr 18 15:18:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    39W /  70W |   9015MiB / 15360MiB |     71%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

 14%|█▍        | 1/7 [01:16<07:36, 76.11s/it]

1.0
100/100 [==============================] - 1s 6ms/step - loss: 40.8898 - accuracy: 0.9360 - categorical_crossentropy: 0.8563
Tue Apr 18 15:20:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    40W /  70W |   9015MiB / 15360MiB |     67%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

...

 71%|███████▏  | 5/7 [06:14<02:29, 74.73s/it]

3.0
100/100 [==============================] - 1s 6ms/step - loss: 33.2006 - accuracy: 0.9394 - categorical_crossentropy: 1.1518
Tue Apr 18 15:24:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    39W /  70W |   9019MiB / 15360MiB |     72%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

 86%|████████▌ | 6/7 [07:28<01:14, 74.53s/it]

3.5
100/100 [==============================] - 1s 6ms/step - loss: 30.9924 - accuracy: 0.9413 - categorical_crossentropy: 1.1306
Tue Apr 18 15:26:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    39W /  70W |   9021MiB / 15360MiB |     69%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

100%|██████████| 7/7 [08:41<00:00, 74.49s/it]


In [58]:
# if if_da:
#   from tensorflow.keras.preprocessing.image import ImageDataGenerator
#   datagen = ImageDataGenerator(
#           rotation_range=10,  # Rotate the image randomly by up to 10 degrees
#           zoom_range=0.1,     # Zoom in or out of the image randomly by up to 10%
#           width_shift_range=0.1,  # Shift the image horizontally by up to 10%
#           height_shift_range=0.1, # Shift the image vertically by up to 10%
#           )
#   datagen.fit(x_train)


In [59]:
# if if_da:
#   from tensorflow import keras
#   from tensorflow.keras import layers

#   # Create a data augmentation stage with horizontal flipping, rotations, zooms
#   data_augmentation = keras.Sequential(
#       [
#           layers.RandomFlip("horizontal"),
#           layers.RandomRotation(0.1),
#           layers.RandomZoom(0.1),
#       ]
#   )

#   input_shape = x_train.shape[1:]
#   classes = 10

#   # Create a tf.data pipeline of augmented images (and their labels)
#   train_augmented = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#   train_augmented = train_augmented.batch(batch_size).map(lambda x, y: (data_augmentation(x), y))

In [60]:
model_variational.summary()

Model: "le_net__variational__large_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_reparameterization_9  (None, 28, 28, 6)        312       
 0 (Conv2DReparameterization                                     
 )                                                               
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 14, 14, 6)        0         
 g2D)                                                            
                                                                 
 conv2d_reparameterization_9  (None, 14, 14, 16)       4832      
 1 (Conv2DReparameterization                                     
 )                                                               
                                                                 
 max_pooling2d_61 (MaxPoolin  (None, 7, 7, 16)         0         
 g2D)                                

In [61]:
# !nvidia-smi

In [62]:
# files.download('prior_scale_1.0_label_noise_0.0_smooth_softmax_3.0_data_augmentation_False.pickle') 

In [63]:
# from google.colab import drive
# drive.mount('/content/drive')

In [64]:
# from google.colab import runtime
# runtime.unassign()